## 04 - Merge With Feature Barcode Info

In [ ]:
from libraries import *
from parameters import *


In [ ]:
os.getcwd()
os.chdir(projectDir)

Please change the file name conventions according to your needs and update the code below accordingly.

In [ ]:
feats = {}

for d in tqdm(sorted([x for x in Path('data').iterdir() if x.is_dir() and x.name.startswith('E')])):
    sample = d.name.replace('_', '-')
    roundlow = sample.split('-')[0].lower()
    chrom = sample.split('-')[1].lower()
    
        
    feat_path = Path(f'data/{sample.replace("-", "_")}/R{roundlow[1:]}_Ch{chrom}_FeatBarc/R{roundlow[1:]}_Ch{chrom}_FeatBarc.csv')
    if feat_path.is_file():
        print(sample)
        feats[sample] = pd.read_csv(feat_path)

In [ ]:
feats_new_barcode = []

for sample, df in tqdm(list(feats.items())):
    df = df.set_index('CRISPR').T
    df['mixhash'] = sample
    df.index = df.index + '-1-' + df.mixhash
    
    feats_new_barcode.append(df)

In [ ]:
feats_new_barcode = pd.concat(feats_new_barcode, axis=0)
del feats_new_barcode['mixhash']


In [ ]:
feats_new_barcode.shape

Replace all 1s with 0s, then compute the total number of crispr reads per cell.

In [ ]:
feats_new_barcode = feats_new_barcode.replace(1, 0)
rowSums = feats_new_barcode.sum(axis=1)
feats_new_barcode = feats_new_barcode[rowSums > 1]
feats_new_barcode.shape

Filter out chimeric reads with a predefined threshold (default 0.2)

In [ ]:
feats_new_barcodePerc = feats_new_barcode.copy()
feats_new_barcodePerc = feats_new_barcodePerc.div(rowSums, axis=0)
m = (feats_new_barcodePerc < par_cutoff_crispr_chimeric_reads)
feats_new_barcode[m] = 0
feats_new_barcode = feats_new_barcode[feats_new_barcode.sum(axis=1) > 0]
feats_new_barcode.shape

In [ ]:
adata = sc.read(par_save_filename_2)
mDF = adata.obs.join(feats_new_barcode,how="inner")
adata = adata[mDF.index,:]
adata.obs = adata.obs.join(feats_new_barcode,how="inner")
adata.uns['feature_barcode_names'] = feats_new_barcode.columns.tolist()

In [ ]:
adata.write(par_save_filename_3)